In [50]:
import numpy as np
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:

## This program plays warhammer by first creating the Players, then creating Models and Weapons, creating Units full of models, assigning the Units to the Players, then doing actions to Models, Units and or Players
class Players:
    def __init__(self, Units, CP, OCP):
        self.Units = Units
        self.CP = CP
        self.OCP = OCP

                             
                                 
                             
                             
    def assign_shooting_targets(self, pot_Targets, mode):#use this for assigning targets
        unit_target_data = {}
        for unit in self.Units:
            eligible_targets = []  # the unit's eligible targets
            unit_in_range = input(f"Is {unit.Name} in range of any enemy? (yes/no): ").strip().lower()
            if unit_in_range in ["yes", "ye", "y"]:
                for pot_target in pot_Targets:
                    target_in_range = input(f"Is {unit.Name} in range of {pot_target.Name}? (yes/no): ").strip().lower()
                    if target_in_range in ["yes", "ye", "y"]:
                        eligible_targets.append(pot_target)
            if unit.Name not in unit_target_data:
                unit_target_data[unit.Name] = {}  # Initialize the dictionary for this unit
            for target in eligible_targets:
                range_to_target = int(input(f"What is the range to {target.Name}? ").strip())
                unit_target_data[unit.Name][target.Name] = {unit.Name: unit, target.Name: target, "Range": range_to_target, "InRangeWeapons": []}
                for warrior in unit.Warriors:
                    for weapon in warrior.Weapons:
                        if weapon.RANGE >= range_to_target:
                            unit_target_data[unit.Name][target.Name]["InRangeWeapons"].append({warrior: weapon})
         

        for unit, targetinfo in unit_target_data.items():
            select_target_from_Names=[]
            select_target_from_finalWoundVal=[]
            select_target_from_WoundMeVal=[]
            select_target_from_ConstructedVal=[]
            for inrangetarget, targetingdata in targetinfo.items():
                for weapondata in targetingdata['InRangeWeapons']:
                    for Warrior, Weapon in weapondata.items():
                         TARGET=targetingdata[inrangetarget]
                         expected_hits=Weapon.A*((7-Weapon.BS)/6)
                         for enemy in TARGET.Warriors:
                             if Weapon.S > enemy.T:
                                 required_roll = 3
                             if Weapon.S >= 2 * enemy.T:
                                 required_roll = 2
                             if Weapon.S == enemy.T:
                                 required_roll = 4
                             if Weapon.S < enemy.T:
                                 required_roll = 5
                             if Weapon.S <= enemy.T / 2:
                                 required_roll = 6
                             prob_num_wound = expected_hits*(7 - required_roll) / 6
                             expected_save = (7-enemy.Sv+Weapon.AP)/6   
                             expected_wounds = prob_num_wound - prob_num_wound*expected_save
                             #print(Weapon.Name, enemy.Name, "Hit:",expected_hits, "woundRl:",prob_num_wound, "save:", expected_save, "finalWound:", expected_wounds)         
                             select_target_from_Names.append(enemy.Name)
                             select_target_from_finalWoundVal.append(expected_wounds)

                             for EWeapon in enemy.Weapons:
                                 print(EWeapon.BS)
                                 Eexpected_hits=EWeapon.A*((7-EWeapon.BS)/6)
                                 if EWeapon.S > Warrior.T:
                                     Erequired_roll = 3
                                 if EWeapon.S >= 2 * Warrior.T:
                                     Erequired_roll = 2
                                 if EWeapon.S == Warrior.T:
                                     Erequired_roll = 4
                                 if EWeapon.S < Warrior.T:
                                     Erequired_roll = 5
                                 if EWeapon.S <= Warrior.T / 2:
                                     Erequired_roll = 6
                                 Eprob_num_wound = Eexpected_hits*(7 - Erequired_roll) / 6
                                 Eexpected_save = (7-Warrior.Sv+EWeapon.AP)/6   
                                 Eexpected_wounds = Eprob_num_wound - Eprob_num_wound*Eexpected_save
                                 print(EWeapon.Name, Warrior.Name, "Hit:",Eexpected_hits, "woundRl:",Eprob_num_wound, "save:", Eexpected_save, "finalWound:", Eexpected_wounds)         
                                 #Eselect_target_from_Names.append(Warrior.Name)
                                 #Eselect_target_from_finalWoundVal.append(Eexpected_wounds)
        return unit_target_data









class Unit:
    def __init__(self, Name, Warriors, ABILITIES, KEYWORDS, DETATCHMENT):
        self.Name=Name
        self.Warriors=Warriors
        self.ABILITIES=ABILITIES
        self.KEYWORDS=KEYWORDS
        self.DETATCHMENT=DETATCHMENT
class Model:
    def __init__(self, Name, M, T, Sv, W, Ld, OC, Weapons, Wargear):
        self.Name=Name
        self.M=M
        self.T=T
        self.Sv=Sv
        self.W=W
        self.Ld=Ld
        self.OC=OC
        self.Weapons=Weapons
        self.Wargear=Wargear
class Weapon:
    def __init__(self, Name, RANGE, A, BS, S, AP, D, Type):
        self.Name=Name
        self.RANGE=RANGE
        self.A=A
        self.BS=BS
        self.S=S
        self.AP=AP
        self.D=D
        self.Type=Type
class Wargear:
    def __init__(self, Name):
        self.Name=Name


#Knowledge
Accelerator_burst_cannon = Weapon("Accelerator burst cannon", RANGE=18, A=4, BS=4, S=6, AP=-1, D=1, Type=None)
Ion_cannon_standard = Weapon("Ion cannon – standard", RANGE=60, A=6.5, BS=4, S=7, AP=-1, D=2, Type=["BLAST"])
Ion_cannon_overcharge = Weapon("Ion cannon – overcharge", RANGE=60, A=6.5, BS=4, S=8, AP=-2, D=3, Type=["BLAST", "HAZARDOUS"])
Railgun = Weapon("Railgun", RANGE=72, A=1, BS=4, S=20, AP=-5, D=9.5, Type=["HEAVY", "DEVASTATING WOUNDS"])
Seeker_missile = Weapon("Seeker missile", RANGE=48, A=1, BS=4, S=14, AP=-3, D=4.5, Type=["ONE SHOT"])
Twin_pulse_carbine = Weapon("Twin pulse carbine", RANGE=20, A=2, BS=4, S=5, AP=0, D=1, Type=["TWIN-LINKED"])
Smart_missile_system = Weapon("Smart missile system", RANGE=30, A=3, BS=4, S=5, AP=0, D=1, Type=["INDIRECT FIRE"])
Hammerhead_Gunship = Model(Name="Hammerhead Gunship",M=10,T=10,Sv=3,W=14,Ld=7,OC=3,Weapons=[Railgun,Seeker_missile,Seeker_missile,Smart_missile_system,Smart_missile_system],Wargear=[])
Hammerhead_Gunship = Unit(Name="Hammerhead Gunship",Warriors=[Hammerhead_Gunship],ABILITIES=["Deadly Demise D3", "For the Greater Good", "Armour Hunter", "Targeting Array"],KEYWORDS=["VEHICLE", "FLY", "HAMMERHEAD GUNSHIP"],DETATCHMENT=["Patient Hunter", "Integrated Command Structure", "Killing Blow"])
Pulse_carbine=Weapon("Pulse carbine",20, 2, 4, 5, 0, 1, None)
Pulse_pistol=Weapon("Pulse pistol",12, 1, 4, 5, 0, 1, ["PISTOL"])
Pulse_rifle=Weapon("Pulse rifle",30, 1, 4, 5, 0, 1, ["RAPID FIRE 1"])
Fireblade_pulse_rifle=Weapon("Fireblade Pulse rifle",30, 1, 3, 5, 0, 2, ["RAPID FIRE 1"])
Support_turrent=Weapon("Support turret",30, 2, 5, 5, 0, 1, ["INDIRECT FIRE", "TWIN LINKED"])
Close_combat_weapon_Fire_Warrior=Weapon("Fire Warrior Close combat weapon", 0, 1, 5, 3, 0, 1, None)
Close_combat_weapon_Cadre_Fireblade=Weapon("Fireblade Close combat weapon", 0, 3, 4, 3, 0, 1, None)
Archon_blast_pistol = Weapon("Archon Blast pistol", RANGE=6, A=1, BS=2, S=8, AP=-3, D=2, Type=["PISTOL"])
Archon_splinter_pistol = Weapon("Archon Splinter pistol", RANGE=12, A=1, BS=2, S=2, AP=0, D=1, Type=["ANTI-INFANTRY 3+", "ASSAULT", "PISTOL"])
Huskblade = Weapon("Huskblade", RANGE=0, A=5, BS=2, S=3, AP=-2, D=2, Type=["ANTI-INFANTRY 3+"])
Blast_pistol = Weapon("Blast pistol", RANGE=6, A=1, BS=3, S=8, AP=-3, D=3, Type=["PISTOL"])
Blaster = Weapon("Blaster", RANGE=18, A=1, BS=3, S=8, AP=-4, D=4.5, Type=["ASSAULT"])
Heat_lance = Weapon("Heat lance", RANGE=18, A=1, BS=3, S=14, AP=-4, D=3.5, Type=["ASSAULT", "MELTA 3"])
Dark_lance = Weapon("Dark lance", RANGE=36, A=1, BS=4, S=12, AP=-3, D=4.5, Type=["HEAVY"])
Shredder = Weapon("Shredder", RANGE=18, A="D6", BS=None, S=6, AP=0, D=1, Type=["ASSAULT", "TORRENT"])
Splinter_cannon = Weapon("Splinter cannon", RANGE=36, A=3, BS=4, S=3, AP=-1, D=2, Type=["ANTI-INFANTRY 3+", "HEAVY", "SUSTAINED HITS 1"])
Splinter_pistol = Weapon("Splinter pistol", RANGE=12, A=1, BS=3, S=2, AP=0, D=1, Type=["ANTI-INFANTRY 3+", "ASSAULT", "PISTOL"])
Splinter_rifle = Weapon("Splinter rifle", RANGE=24, A=2, BS=3, S=2, AP=0, D=1, Type=["ANTI-INFANTRY 3+", "ASSAULT"])
Sybarite_weapon = Weapon("Sybarite weapon", RANGE=0, A=3, BS=3, S=3, AP=-1, D=1, Type=["ANTI-INFANTRY 3+"])
Close_combat_weapon_Kabalite = Weapon("Kabalite Close combat weapon", 0, 2, 3, 3, 0, 1, [])
Twin_Drukhari_haywire_blaster = Weapon("Twin Drukhari haywire blaster", RANGE=24, A=2, BS=4, S=3, AP=-1, D=3, Type=["ANTI-VEHICLE 4+", "DEVASTATING WOUNDS", "TWIN-LINKED"])
Twin_heat_lance = Weapon("Twin heat lance", RANGE=18, A=1, BS=4, S=14, AP=-4, D=3.5, Type=["ASSAULT", "MELTA 3", "TWIN-LINKED"])
Twin_splinter_cannon = Weapon("Twin splinter cannon", RANGE=36, A=3, BS=4, S=3, AP=-1, D=2, Type=["ANTI-INFANTRY 3+", "SUSTAINED HITS 1", "TWIN-LINKED"])
Stinger_pod = Weapon("Stinger pod", RANGE=24, A=7, BS=4, S=5, AP=0, D=1, Type=[])
Twin_liquifier_gun = Weapon("Twin liquifier gun", RANGE=12, A=3.5, BS=None, S=4, AP=-1, D=1, Type=["TORRENT", "TWIN-LINKED"])
Chain_flails = Weapon("Chain-flails", RANGE=0, A=8, BS=3, S=6, AP=0, D=1, Type=[])
Macro_scalpel = Weapon("Macro-scalpel", RANGE=0, A=5, BS=3, S=7, AP=-1, D=2, Type=[])
Talos_gauntlet = Weapon("Talos gauntlet", RANGE=0, A=5, BS=4, S=8, AP=-2, D=3, Type=[])
Talos_ichor_injector = Weapon("Talos ichor injector", RANGE=0, A=1, BS=3, S=7, AP=-1, D=2, Type=["EXTRA ATTACKS"])
Agoniser = Weapon("Agoniser", RANGE=0, A=4, BS=3, S=3, AP=-1, D=1, Type=["ANTI-INFANTRY 3+"])
Bladevane = Weapon("Bladevanes", RANGE=0, A=3, BS=3, S=4, AP=0, D=1, Type=[])
Guardian_drone=Wargear("Guardian Drone")
Marker_drone=Wargear("Marker Drone")
Shield_drone=Wargear("Shield Drone")
Fire_Warrior_Pulse_rifle=Model("Fire Warrior", 6, 3, 4, 1, 7, 2, [Pulse_rifle, Pulse_pistol, Close_combat_weapon_Fire_Warrior],[])
Fire_Warrior_Shasui_Pulse_rifle=Model("Fire Warrior Shasui", 6, 3, 4, 1, 7, 2, [Pulse_rifle, Pulse_pistol, Close_combat_weapon_Fire_Warrior, Support_turrent], [Guardian_drone])
Cadre_Fireblade_Pulse_rifle=Model("Cadre Fireblade", 6, 3, 4, 3, 7, 1, [Pulse_rifle, Pulse_pistol, Close_combat_weapon_Cadre_Fireblade], [Shield_drone, "Volly Fire", "Crack Shot"])
Reaver_Splinter_rifle = Model(Name="Reaver", M=16, T=4, Sv=4, W=2, Ld=6, OC=2, Weapons=[Splinter_rifle,Splinter_pistol,Bladevane], Wargear=[])
Reaver_Arena_Champion = Model("Reaver Arena Champion", M=16, T=4, Sv=4, W=2, Ld=6, OC=2, Weapons=[Splinter_rifle,Splinter_pistol,Bladevane,Agoniser], Wargear=[])
Reaver_Heat_lance = Model(Name="Reaver", M=16, T=4, Sv=4, W=2, Ld=6, OC=2, Weapons=[Heat_lance,Splinter_pistol,Bladevane], Wargear=[])
Archon = Model(Name="Archon",M=8,T=3,Sv=4,W=4,Ld=6,OC=1,Weapons=[Archon_splinter_pistol, Huskblade],Wargear=["Overload", "Devious Mastermind (Aura)"])
Kabalite_Warrior_Splinter_rifle = Model("Kabalite Warrior", M=8, T=3, Sv=4, W=1, Ld=6, OC=2, Weapons=[Splinter_rifle,Close_combat_weapon_Kabalite], Wargear=[])
Sybarite_Splinter_rifle = Model("Sybarite", M=8, T=3, Sv=4, W=1, Ld=6, OC=2, Weapons=[Splinter_rifle,Sybarite_weapon], Wargear=[])
Kabalite_Warrior_Blaster = Model("Kabalite Warrior", M=8, T=3, Sv=4, W=1, Ld=6, OC=2, Weapons=[Blaster,Close_combat_weapon_Kabalite], Wargear=[])
Kabalite_Warrior_Dark_lance = Model("Kabalite Warrior", M=8, T=3, Sv=4, W=1, Ld=6, OC=2, Weapons=[Dark_lance,Close_combat_weapon_Kabalite], Wargear=[])
Kabalite_Warrior_Shredder = Model("Kabalite Warrior", M=8, T=3, Sv=4, W=1, Ld=6, OC=2, Weapons=[Shredder,Close_combat_weapon_Kabalite], Wargear=[])
Kabalite_Warrior_Splinter_cannon = Model("Kabalite Warrior", M=8, T=3, Sv=4, W=1, Ld=6, OC=2, Weapons=[Splinter_cannon,Close_combat_weapon_Kabalite], Wargear=[])
Talos_Engine_Heat_lance_Twin_liquifier_gun_Chain_flails = Model(Name="Talos", M=7, T=7, Sv=3, W=7, Ld=6, OC=2, Weapons=[Twin_heat_lance,Twin_liquifier_gun,Chain_flails], Wargear=[])
Talos=Unit("Talos Engine", [Talos_Engine_Heat_lance_Twin_liquifier_gun_Chain_flails],["Deadly Demise 1","Feel No Pain 5+","Power From Pain","Pain Engine"],["MONSTER", "HAEMONCULUS COVENS", "FLY", "AELDARI", "TALOS"],["Realspace Raiders", "Callous Competition", "Rain Of Cruelty"])
Kabalite_Warriors_Archon=Unit("Kabalite Warriors with Archon", [Archon,Sybarite_Splinter_rifle,Kabalite_Warrior_Blaster,Kabalite_Warrior_Dark_lance,Kabalite_Warrior_Shredder,Kabalite_Warrior_Splinter_cannon,Kabalite_Warrior_Splinter_rifle,Kabalite_Warrior_Splinter_rifle,Kabalite_Warrior_Splinter_rifle,Kabalite_Warrior_Splinter_rifle,Kabalite_Warrior_Splinter_rifle],["Power From Pain","Sadistic Raiders""Phantasm Grenade Launcher"],["INFANTRY", "KABAL", "BATTLELINE", "AELDARI", "KABALITE WARRIORS"],["Realspace Raiders", "Callous Competition", "Rain Of Cruelty", "Raid and Pillage"])
Reavers=Unit("Reavers",[Reaver_Arena_Champion,Reaver_Heat_lance,Reaver_Splinter_rifle],["Power From Pain","Eviscerating Fly-by","Cluster Caltrops","Grav-talon"],["MOUNTED", "WYCH CULT", "FLY", "AELDARI", "REAVERS"],["Realspace Raiders","Callous Competition","Rain Of Cruelty"])
Strike_Team_Cadre_Fireblade=Unit("Strike Team Cadre Fireblade", [Cadre_Fireblade_Pulse_rifle, Fire_Warrior_Shasui_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle],["For the Greater Good","Suppression Volley","DS8 Support Turret"],[ "INFANTRY", "BATTLELINE", "GRENADES", "MARKERLIGHT", "FIRE WARRIOR", "STRIKE TEAM"],["Patient Hunter", "Integrated Command Structure", "Killing Blow", "Markerlight Precision"])
Strike_Team=Unit("Strike Team", [Fire_Warrior_Shasui_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle],["For the Greater Good","Suppression Volley","DS8 Support Turret"],[ "INFANTRY", "BATTLELINE", "GRENADES", "MARKERLIGHT", "FIRE WARRIOR", "STRIKE TEAM"],["Patient Hunter", "Integrated Command Structure", "Killing Blow", "Markerlight Precision"])
Player1=Players([Strike_Team_Cadre_Fireblade, Strike_Team, Hammerhead_Gunship],0,0)
Player2=Players([Kabalite_Warriors_Archon, Talos, Reavers],0,0)
target_data = Player1.assign_shooting_targets(Player2.Units, None)
print("Target Data:", target_data)
kabalytetest=Close_combat_weapon_Kabalite.RANGE


Is Strike Team Cadre Fireblade in range of any enemy? (yes/no):  y
Is Strike Team Cadre Fireblade in range of Kabalite Warriors with Archon? (yes/no):  y
Is Strike Team Cadre Fireblade in range of Talos Engine? (yes/no):  n
Is Strike Team Cadre Fireblade in range of Reavers? (yes/no):  n
What is the range to Kabalite Warriors with Archon?  12
Is Strike Team in range of any enemy? (yes/no):  n
Is Hammerhead Gunship in range of any enemy? (yes/no):  n


2
Archon Splinter pistol Cadre Fireblade Hit: 0.8333333333333334 woundRl: 0.2777777777777778 save: 0.5 finalWound: 0.1388888888888889
2
Huskblade Cadre Fireblade Hit: 4.166666666666667 woundRl: 2.0833333333333335 save: 0.16666666666666666 finalWound: 1.7361111111111112
3
Splinter rifle Cadre Fireblade Hit: 1.3333333333333333 woundRl: 0.4444444444444444 save: 0.5 finalWound: 0.2222222222222222
3
Sybarite weapon Cadre Fireblade Hit: 2.0 woundRl: 1.0 save: 0.3333333333333333 finalWound: 0.6666666666666667
3
Blaster Cadre Fireblade Hit: 0.6666666666666666 woundRl: 0.5555555555555555 save: -0.16666666666666666 finalWound: 0.648148148148148
3
Kabalite Close combat weapon Cadre Fireblade Hit: 1.3333333333333333 woundRl: 0.6666666666666666 save: 0.5 finalWound: 0.3333333333333333
4
Dark lance Cadre Fireblade Hit: 0.5 woundRl: 0.4166666666666667 save: 0.0 finalWound: 0.4166666666666667
3
Kabalite Close combat weapon Cadre Fireblade Hit: 1.3333333333333333 woundRl: 0.6666666666666666 save: 0.5 f

TypeError: unsupported operand type(s) for -: 'int' and 'NoneType'

In [37]:

## This program plays warhammer by first creating the Players, then creating Models and Weapons, creating Units full of models, assigning the Units to the Players, then doing actions to Models, Units and or Players

##Create Player
#class Player(Command_Points, Units):

class Players:
    def __init__(self, Units, CP, OCP):
        self.Units = Units
        self.CP = CP
        self.OCP = OCP

    def assign_shooting_targets(self, pot_Targets, mode):
        unit_target_data={}
        
        
        ### for each unit, loop through the targets, and append the targets that are in range
        for unit in self.Units:
            eligible_targets = []            ###the units eligible target
            unit_in_range = input(f"Is {unit.Name} in range of any enemy? (yes/no): ").strip().lower()
            
            if unit_in_range in ["yes", "ye", "y"]:
                for pot_target in pot_Targets:
                    target_in_range = input(f"Is {unit.Name} in range of {pot_target.Name}? (yes/no): ").strip().lower()
                    if target_in_range in ["yes", "ye", "y"]:
                        eligible_targets.append(pot_target)           

            #For the targets in range, get the ranges and save them
            
            for target in eligible_targets:
                range_to_target = int(input(f"What is the range to {target.Name}? ").strip())
                unit_target_data[target.Name] = {"Unit": unit.Name, "Target":target.Name, "Range": range_to_target, "InRangeWeapons":[]}

                # Iterate through Warriors in the Unit to print their weapons
                for warrior in unit.Warriors:
                    for weapon in warrior.Weapons:
                        if weapon.RANGE>=range_to_target:
                            unit_target_data[target.Name]["InRangeWeapons"].append({warrior.Name:weapon.Name})

        return unit_target_data

   
            

class Unit:
    def __init__(self, Name, Warriors, ABILITIES, KEYWORDS, DETATCHMENT):
        self.Name=Name
        self.Warriors=Warriors
        self.ABILITIES=ABILITIES
        self.KEYWORDS=KEYWORDS
        self.DETATCHMENT=DETATCHMENT
        
##Create a model in Unit
class Model:
    def __init__(self, Name, M, T, Sv, W, Ld, OC, Weapons, Wargear):
        self.Name=Name
        self.M=M
        self.T=T
        self.Sv=Sv
        self.W=W
        self.Ld=Ld
        self.OC=OC
        self.Weapons=Weapons
        self.Wargear=Wargear





##Create Weapons
class Weapon:
    def __init__(self, Name, RANGE, A, BS, S, AP, D, Type):
        self.Name=Name
        self.RANGE=RANGE
        self.A=A
        self.BS=BS
        self.S=S
        self.AP=AP
        self.D=D
        self.Type=Type

class Wargear:
    def __init__(self, Name):
        self.Name=Name


        






Accelerator_burst_cannon = Weapon("Accelerator burst cannon", RANGE=18, A=4, BS=4, S=6, AP=-1, D=1, Type=None)
Ion_cannon_standard = Weapon("Ion cannon – standard", RANGE=60, A="D6+3", BS=4, S=7, AP=-1, D=2, Type=["BLAST"])
Ion_cannon_overcharge = Weapon("Ion cannon – overcharge", RANGE=60, A="D6+3", BS=4, S=8, AP=-2, D=3, Type=["BLAST", "HAZARDOUS"])
Railgun = Weapon("Railgun", RANGE=72, A=1, BS=4, S=20, AP=-5, D="D6+6", Type=["HEAVY", "DEVASTATING WOUNDS"])
Seeker_missile = Weapon("Seeker missile", RANGE=48, A=1, BS=4, S=14, AP=-3, D="D6+1", Type=["ONE SHOT"])
Twin_pulse_carbine = Weapon("Twin pulse carbine", RANGE=20, A=2, BS=4, S=5, AP=0, D=1, Type=["TWIN-LINKED"])
Smart_missile_system = Weapon("Smart missile system", RANGE=30, A=3, BS=4, S=5, AP=0, D=1, Type=["INDIRECT FIRE"])

# Define the Hammerhead Gunship Model
Hammerhead_Gunship = Model(Name="Hammerhead Gunship",M=10,T=10,Sv=3,W=14,Ld=7,OC=3,Weapons=[Railgun,Seeker_missile,Seeker_missile,Smart_missile_system,Smart_missile_system],Wargear=[])

# Define the Hammerhead Gunship Unit
Hammerhead_Gunship = Unit(Name="Hammerhead Gunship",Warriors=[Hammerhead_Gunship],ABILITIES=["Deadly Demise D3", "For the Greater Good", "Armour Hunter", "Targeting Array"],KEYWORDS=["VEHICLE", "FLY", "HAMMERHEAD GUNSHIP"],DETATCHMENT=["Patient Hunter", "Integrated Command Structure", "Killing Blow"])






Pulse_carbine=Weapon("Pulse carbine",20, 2, 4, 5, 0, 1, None)
Pulse_pistol=Weapon("Pulse pistol",12, 1, 4, 5, 0, 1, ["PISTOL"])
Pulse_rifle=Weapon("Pulse rifle",30, 1, 4, 5, 0, 1, ["RAPID FIRE 1"])
Fireblade_pulse_rifle=Weapon("Pulse rifle",30, 1, 3, 5, 0, 2, ["RAPID FIRE 1"])
Support_turrent=Weapon("Support turret",30, 2, 5, 5, 0, 1, ["INDIRECT FIRE", "TWIN LINKED"])
Close_combat_weapon_Fire_Warrior=Weapon("Close combat weapon", 0, 1, 5, 3, 0, 1, None)
Close_combat_weapon_Cadre_Fireblade=Weapon("Close combat weapon", 0, 3, 4, 3, 0, 1, None)
Guardian_drone=Wargear("Guardian Drone")
Marker_drone=Wargear("Marker Drone")
Shield_drone=Wargear("Shield Drone")
Fire_Warrior_Pulse_rifle=Model("Fire Warrior", 6, 3, 4, 1, 7, 2, [Pulse_rifle, Pulse_pistol, Close_combat_weapon_Fire_Warrior],[])

Fire_Warrior_Shasui_Pulse_rifle=Model("Fire Warrior Shasui", 6, 3, 4, 1, 7, 2, [Pulse_rifle, Pulse_pistol, Close_combat_weapon_Fire_Warrior, Support_turrent], [Guardian_drone])
Cadre_Fireblade_Pulse_rifle=Model("Cadre Fireblade", 6, 3, 4, 3, 7, 1, [Pulse_rifle, Pulse_pistol, Close_combat_weapon_Cadre_Fireblade, Support_turrent], [Shield_drone, "Volly Fire", "Crack Shot"])

Strike_Team_Cadre_Fireblade=Unit("Strike Team Cadre Fireblade", [Cadre_Fireblade_Pulse_rifle, Fire_Warrior_Shasui_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle],["For the Greater Good","Suppression Volley","DS8 Support Turret"],[ "INFANTRY", "BATTLELINE", "GRENADES", "MARKERLIGHT", "FIRE WARRIOR", "STRIKE TEAM"],["Patient Hunter", "Integrated Command Structure", "Killing Blow", "Markerlight Precision"])
Strike_Team=Unit("Strike Team", [Fire_Warrior_Shasui_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle,Fire_Warrior_Pulse_rifle],["For the Greater Good","Suppression Volley","DS8 Support Turret"],[ "INFANTRY", "BATTLELINE", "GRENADES", "MARKERLIGHT", "FIRE WARRIOR", "STRIKE TEAM"],["Patient Hunter", "Integrated Command Structure", "Killing Blow", "Markerlight Precision"])
Player1=Players([Strike_Team_Cadre_Fireblade, Strike_Team],0,0)

target_data = Player1.assign_shooting_targets([Strike_Team, Hammerhead_Gunship], None)
print("Target Data:", target_data)


Is Strike Team Cadre Fireblade in range of any enemy? (yes/no):  y
Is Strike Team Cadre Fireblade in range of Strike Team? (yes/no):  y
Is Strike Team Cadre Fireblade in range of Hammerhead Gunship? (yes/no):  n
What is the range to Strike Team?  29
Is Strike Team in range of any enemy? (yes/no):  n


Target Data: {'Strike Team': {'Unit': 'Strike Team Cadre Fireblade', 'Target': 'Strike Team', 'Range': 29, 'InRangeWeapons': [{'Cadre Fireblade': 'Pulse rifle'}, {'Cadre Fireblade': 'Support turret'}, {'Fire Warrior Shasui': 'Pulse rifle'}, {'Fire Warrior Shasui': 'Support turret'}, {'Fire Warrior': 'Pulse rifle'}, {'Fire Warrior': 'Pulse rifle'}, {'Fire Warrior': 'Pulse rifle'}, {'Fire Warrior': 'Pulse rifle'}, {'Fire Warrior': 'Pulse rifle'}, {'Fire Warrior': 'Pulse rifle'}, {'Fire Warrior': 'Pulse rifle'}, {'Fire Warrior': 'Pulse rifle'}, {'Fire Warrior': 'Pulse rifle'}]}}


In [ ]:
import numpy as np

# Constants for Wound and Hit charts
Wound_Chart = {
    0: [7, 7, 7, 7, 7, 7, 7, 7, 7, 7],
    1: [4, 5, 6, 6, 7, 7, 7, 7, 7, 7],
    2: [3, 4, 5, 6, 6, 7, 7, 7, 7, 7],
    3: [2, 3, 4, 5, 6, 6, 7, 7, 7, 7],
    4: [2, 2, 3, 4, 5, 6, 6, 7, 7, 7],
    5: [2, 2, 2, 3, 4, 5, 6, 6, 7, 7],
    6: [2, 2, 2, 2, 3, 4, 5, 6, 6, 7],
    7: [2, 2, 2, 2, 2, 3, 4, 5, 6, 6],
    8: [2, 2, 2, 2, 2, 2, 3, 4, 5, 6],
    9: [2, 2, 2, 2, 2, 2, 2, 3, 4, 5],
    10: [2, 2, 2, 2, 2, 2, 2, 2, 3, 4]
}

Hit_Chart = {
    1: [4, 4, 5, 5, 5, 5, 5, 5, 5, 5],
    2: [3, 4, 4, 5, 5, 5, 5, 5, 5, 5],
    3: [3, 3, 4, 4, 4, 4, 5, 5, 5, 5],
    4: [3, 3, 3, 4, 4, 4, 4, 4, 5, 5],
    5: [3, 3, 3, 3, 4, 4, 4, 4, 4, 4],
    6: [3, 3, 3, 3, 3, 4, 4, 4, 4, 4],
    7: [3, 3, 3, 3, 3, 3, 4, 4, 4, 4],
    8: [3, 3, 3, 3, 3, 3, 3, 4, 4, 4],
    9: [3, 3, 3, 3, 3, 3, 3, 3, 4, 4],
    10: [3, 3, 3, 3, 3, 3, 3, 3, 3, 4]
}

class Weapon:
    def __init__(self, name, range, strength, ap, type):
        self.name = name
        self.range = range
        self.strength = strength
        self.ap = ap
        self.type = type

class MeleeWeapon:
    def __init)__(

class Equipment:
    def __init__(WSmodify, BSmodify, Smodify, Tmodify, Wmodify, Imodify, Amodify, Ldmodify, Svmodify):
        self.WSmodify = WSmodify
        self.BSmodify = BSmodify
        self.Smodify = Smodify
        self.Tmodify = Tmodify
        self.Wmodify = Wmodify
        self.Imodify = Imodify
        self.Amodify = Amodify
        self.Ldmodify = Ldmodify
        self.Svmodify = Svmodify
        

class Model:
    def __init__(self, WS, BS, S, T, W, I, A, Ld, Sv, wargear):
        self.WS = WS  
        self.BS = BS  
        self.S = S
        self.T = T
        self.W = W
        self.I = I 
        self.A = A
        self.Ld = Ld
        self.Sv = Sv
        self.wargear = wargear
        
    def take_wounds(self, wounds):              
        self.W -= wounds
    
    def shoot(self, target, weapon_name): 
        weapon = self.wargear[weapon_name]
        Ehit = (6 - self.BS)/6
        Ewound = (7 - Wound_Chart[weapon.strength][target.T])/6
        EArmorPierce = (target.Sv - 1)/6 
        if weapon.ap > target.Sv:
            EArmorPierce = 1
        Edamage = Ehit * Ewound * EArmorPierce
        return Edamage

    def melee(self, target):                        
        Ehit = (7 - Hit_Chart[self.WS][target.WS])/6 
        Ewound = (7 - Wound_Chart[self.S][target.T])/6
        Edamage = Ehit * Ewound
        return Edamage
    
    def snapshot(self, target, weapon_name):               
        weapon = self.wargear[weapon_name]
        Ehit = 1/6 
        Ewound = (7 - Wound_Chart[weapon.strength][target.T])/6
        EArmorPierce = (target.Sv - 1)/6 
        if weapon.ap > target.Sv:
            EArmorPierce = 1
        Edamage = Ehit * Ewound * EArmorPierce
        return Edamage

class Unit:  
    def __init__(self, Max_Move, latitude, longitude, radius, warriors, Special_Rules):       
        self.radius = radius
        self.longitude = longitude
        self.latitude = latitude
        self.Max_Move = Max_Move
        self.warriors = warriors
        self.Special_Rules = Special_Rules
        
    def move(self, dX, dY):
        start = np.array([self.latitude, self.longitude])
        self.latitude += dX
        self.longitude += dY
        finish = np.array([self.latitude, self.longitude])
        diff = finish - start
        sq_diff = diff**2
        sum_sq_diff = np.sum(sq_diff)
        moved = np.sqrt(sum_sq_diff)
        return moved
   
    def distance(self, target):                                                          
        point1 = np.array([self.latitude, self.longitude])
        point2 = np.array([target.latitude, target.longitude])
        diff = point2 - point1
        sq_diff = diff**2
        sum_sq_diff = np.sum(sq_diff)
        distance = np.sqrt(sum_sq_diff)
        return distance


# Define your wargear as instances of the Weapon class
Pulse_Rifle = Weapon('Pulse Rifle', 30, 5, 5, 'Rapid Fire 2')
Markerlight = Weapon('Markerlight', 36, 0, 0,'Heavy 1, Target Aquired')  
Defensive_Grenade = Weapon('Defensive_Grenade', 8, 1, 0, 'Assault 1, Blast, Blind')  



# Create instances of Model with appropriate wargear
Fire_Warrior_Pulse_Rifle = Model(2, 3, 3, 3, 1, 2, 1, 7, 4, {'Pulse Rifle': Pulse_Rifle, 'Defensive Grenade':Defensive_Grenade})
Fire_Warrior_Shasui = Model(2, 3, 3, 3, 1, 2, 1, 7, 4, {'Pulse Rifle': Pulse_Rifle, 'Markerlight': Markerlight, 'Defensive Grenade':Defensive_Grenade})

# Example usage
Strike_Team_Warriors = [
    Fire_Warrior_Shasui,
    Fire_Warrior_Pulse_Rifle,
    Fire_Warrior_Pulse_Rifle,
    Fire_Warrior_Pulse_Rifle,
    Fire_Warrior_Pulse_Rifle,
    Fire_Warrior_Pulse_Rifle,
    Fire_Warrior_Pulse_Rifle,
    Fire_Warrior_Pulse_Rifle,
    Fire_Warrior_Pulse_Rifle,
    Fire_Warrior_Pulse_Rifle
]
Strike_Team = Unit(6, 2, 2, 1, Strike_Team_Warriors,'Supporting Fire')
Target_Model = Model(2, 3, 3, 3, 1, 2, 1, 7, 4, {'Pulse Rifle': Pulse_Rifle})
Target_Warriors = [Target_Model, Target_Model, Target_Model]
Target_Unit = Unit(6, 9, 9, 1, Target_Warriors, 'None')

# Example combat scenarios
print(f"Fire Warrior Shasui  damage: {Fire_Warrior_Shasui.shoot(Target_Model, 'Defensive Grenade')}")




In [28]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
%matplotlib inline

class Model:
    def __init__(self, WS, BS, S, T, W, I, A, Ld, Sv, Wargear):       
        self.WS = WS  
        self.BS = BS  
        self.S = S
        self.T = T
        self.W = W
        self.I = I 
        self.S = A
        self.Ld = Ld
        self.Sv = Sv
        self.Unit = Unit
        self.Wargear = Wargear
        
    def take_wounds(self, wounds):              
        self.W = self.W - wounds
    

    def shoot(self, target, weapon_name): 
        weapon = self.Wargear[weapon_name]
        Ehit = (6 - self.BS)/6
        Ewound = (7 - Wound_Chart[weapon['S']][target.T])/6
        EArmorPierce= (target.Sv-1)/6 
        if weapon['AP'] > target.Sv:
            EArmorPierce = 1
        Edamage = Ehit * Ewound * EArmorPierce
        return Edamage



    
    def melee(self, target):                        
        Ehit = (7 - Hit_Chart[self.WS][target.WS])/6 
        Ewound = (7 - Wound_Chart[self.S][target.T])/6
        Edamage = Ehit*Ewound
        return Edamage
    
    def snapshot (self, target, weapon):               
        Ehit = 1/6 
        Ewound = (7 - Wound_Chart[self.Wargear['S']][target.T])/6
        EArmorPierce= (target.Sv-1)/6 
        if self.Wargear['AP'] > target.Sv:
            EArmorPierce = 1
        Edamage = Ehit*Ewound*EArmorPierce
        return Edamage

class Unit:  
    def __init__(self, Max_Move, latitude, longitude, radius, warriors):       
        self.radius=radius
        self.longitude=longitude
        self.latitude=latitude
        self.Max_Move=Max_Move
        self.warriors=warriors
        
    def move(self, dX, dY):
        start=np.array([self.latitude, self.longitude])
        self.latitude=self.latitude+dX
        self.longitude=self.longitude+dY
        finish=np.array([self.latitude, self.longitude])
        diff = finish - start
        sq_diff = diff**2
        sum_sq_diff = np.sum(sq_diff)
        moved = np.sqrt(sum_sq_diff)
        return moved
   
    def distance(self,target):                                                          
        point1 = np.array([self.latitude, self.longitude])
        point2 = np.array([target.latitude, target.longitude])
        diff = point2 - point1
        sq_diff = diff**2
        sum_sq_diff = np.sum(sq_diff)
        distance = np.sqrt(sum_sq_diff)
        print(f" {point1}")  
        print(f" {point2}")  
        return distance


Wound_Chart = {0:[0,0,0,0,0,0,0,0,0,0], 1:[4,5,6,6,7,7,7,7,7,7], 2:[3,4,5,6,6,7,7,7,7,7], 3:[2,3,4,5,6,6,7,7,7,7], 4:[2,2,3,4,5,6,6,7,7,7], 5:[2,2,2,3,4,5,6,6,7,7], 6:[2,2,2,2,3,4,5,6,6,7], 7:[2,2,2,2,2,3,4,5,6,6], 8:[2,2,2,2,2,2,3,4,5,6], 9:[2,2,2,2,2,2,2,3,4,5], 10:[2,2,2,2,2,2,2,2,3,4]}
Hit_Chart = {1:[4,4,5,5,5,5,5,5,5,5],2:[3,4,4,5,5,5,5,5,5,5],3:[3,3,4,4,4,4,5,5,5,5],4:[3,3,3,4,4,4,4,4,5,5],5:[3,3,3,3,4,4,4,4,4,4],6:[3,3,3,3,3,4,4,4,4,4],7:[3,3,3,3,3,3,4,4,4,4],8:[3,3,3,3,3,3,3,4,4,4],9:[3,3,3,3,3,3,3,3,4,4],10:[3,3,3,3,3,3,3,3,3,4]}

Pulse_Rifle = {'Range':30, 'S':5, 'AP':5, 'Type':'Rapid Fire'} 
Markerlight = {'Range':36, 'Type':'Heavy 1'}
Fire_Warrior_Shasui_Kit = [Pulse_Rifle, Markerlight]
Fire_Warrior_Pulse_Rifle = Model(2, 3, 3, 3, 1, 2, 1, 7, 4, Pulse_Rifle)
Fire_Warrior_Shasui = Model(2, 3, 3, 3, 1, 2, 1, 7, 4, Fire_Warrior_Shasui_Kit)

Strike_Team_Warriors= [Fire_Warrior_Shasui,Fire_Warrior_Pulse_Rifle,Fire_Warrior_Pulse_Rifle,Fire_Warrior_Pulse_Rifle,Fire_Warrior_Pulse_Rifle,Fire_Warrior_Pulse_Rifle,Fire_Warrior_Pulse_Rifle,Fire_Warrior_Pulse_Rifle,Fire_Warrior_Pulse_Rifle,Fire_Warrior_Pulse_Rifle]
Strike_Team =Unit(6,2,2,1,Strike_Team_Warriors)
Target_Model = Model(2,3,3,3,1,2,1,7,4, Pulse_Rifle)
Target_Warriors=[Target_Model,Target_Model,Target_Model]
Target_Unit=Unit(6,9,9,1,Target_Warriors)                        







print(f" x: {Strike_Team.latitude}")  
print(f" y: {Strike_Team.longitude}")
print(f" distance: {Strike_Team.distance(Target_Unit)}")
print(f" moved: {Strike_Team.move(1,1)}")                      
print(f"new x: {Strike_Team.latitude}")
print(f"new y: {Strike_Team.longitude}")
print(f" new distance: {Strike_Team.distance(Target_Unit)}")
print(Wound_Chart[2][4]) 

print(f" {Fire_Warrior_Shasui.shoot(Target_Model,Pulse_Rifle)}")






 x: 2
 y: 2
 [2 2]
 [9 9]
 distance: 9.899494936611665
 moved: 1.4142135623730951
new x: 3
new y: 3
 [3 3]
 [9 9]
 new distance: 8.48528137423857
6


TypeError: list indices must be integers or slices, not dict

Pulse_Rifle = {'Range':30, 'S':5, 'AP':5, 'Type':'Rapid Fire'}
print(Pulse_Rifle['S'])

In [83]:

print(dir(np))


['ALLOW_THREADS', 'BUFSIZE', 'CLIP', 'DataSource', 'ERR_CALL', 'ERR_DEFAULT', 'ERR_IGNORE', 'ERR_LOG', 'ERR_PRINT', 'ERR_RAISE', 'ERR_WARN', 'FLOATING_POINT_SUPPORT', 'FPE_DIVIDEBYZERO', 'FPE_INVALID', 'FPE_OVERFLOW', 'FPE_UNDERFLOW', 'False_', 'Inf', 'Infinity', 'MAXDIMS', 'MAY_SHARE_BOUNDS', 'MAY_SHARE_EXACT', 'NAN', 'NINF', 'NZERO', 'NaN', 'PINF', 'PZERO', 'RAISE', 'RankWarning', 'SHIFT_DIVIDEBYZERO', 'SHIFT_INVALID', 'SHIFT_OVERFLOW', 'SHIFT_UNDERFLOW', 'ScalarType', 'True_', 'UFUNC_BUFSIZE_DEFAULT', 'UFUNC_PYVALS_NAME', 'WRAP', '_CopyMode', '_NoValue', '_UFUNC_API', '__NUMPY_SETUP__', '__all__', '__builtins__', '__cached__', '__config__', '__deprecated_attrs__', '__dir__', '__doc__', '__expired_functions__', '__file__', '__former_attrs__', '__future_scalars__', '__getattr__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_add_newdoc_ufunc', '_builtins', '_distributor_init', '_financial_names', '_get_promotion_state', '_globals', '_int_extended_ms